In [ ]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15979 sha256=39598a99affc141fb4c2766e999f58fbba02308e07cfacd0d98abc1629f8d4c1
  Stored in directory: /root/.cache/pip/wheels/89/2d/c2/abe6bcfde6bce368c00ecd73310c11edb672c3eda09a090cfa
Successfully built pyngrok


In [ ]:
!pip install --upgrade streamlit
import streamlit as st

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.6 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=3d53d149d58bc993efbeb239de27e186082edf3fedb07

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/DS440/Data/image

/content/gdrive/.shortcut-targets-by-id/1HuqlSiQQRulpqyyP747KMhMiCeBTkfqs/DS440/Data/image


In [ ]:
!ls

12345.jpg	    best_model.h5  features.pkl  model.png
16641184431457.jpg  captions.txt   Images	 tokenizer.pkl


In [ ]:
%%writefile app.py
import os   # handling the files
import pickle # storing numpy features
import numpy as np
from tqdm.notebook import tqdm # how much data is process till now
from keras.models import load_model
import cv2

from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input # extract features from image data.
from tensorflow.keras.preprocessing.image import load_img , img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input , Dense , LSTM , Embedding , Dropout , add

import streamlit as st
import pickle
import numpy as np
import pandas as pd
import string

Navigation = {"page_title":"ImageCaptionGenerator.io","page_icon":"👽","layout":"centered"}
st.set_page_config(**Navigation)



def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate caption for an image
def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break
    return in_text

from PIL import Image
import matplotlib.pyplot as plt
# def generate_caption(image_name):
#     # load the image
#     # image_name = "1001773457_577c3a7d70.jpg"
#     image_id = "1"

#     # predict the caption
#     y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
#     # print('--------------------Predicted--------------------')
#     # print(y_pred)
#     # plt.imshow(image)



def load_tokenizer():
    with open('tokenizer.pkl', 'rb') as handle:
        tokenizer = pickle.load(handle)
    return tokenizer

def load_image(img):
    im = Image.open(img)
    image = np.array(im)
    return image

  
tokenizer = load_tokenizer()
model = load_model("/content/gdrive/My Drive/DS440/Data/image/best_model.h5")
max_length = 35

def main():
    st.header("ImageCaptionGenerator")

    st.write("""### Please upload your image:""")


    # construct text box
    input_buffer = st.file_uploader("Upload an image", type=("png", "jpg"))

    button = st.button("Generate")

    if button:
      if input_buffer:
        # extract features from image
        features = {}
        image_name = input_buffer
        # Load vgg16 Model
        model_vgg = VGG16()
        # restructure model
        model_vgg = Model(inputs = model_vgg.inputs , outputs = model_vgg.layers[-2].output)
        # load the image from file
        image = load_img(image_name, target_size=(224, 224))
        # convert image pixels to numpy array
        image = img_to_array(image)
        # reshape data for model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # preprocess image for vgg
        image = preprocess_input(image)
        # extract features
        feature = model_vgg.predict(image, verbose=0)
        # get image ID
        image_id = "1"
        # image_name.split('.')[0]
        # store feature
        features[image_id] = feature

        y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
        st.subheader(y_pred)
        uploaded_img = load_image(image_name)
        st.image(uploaded_img)
        # else:
        #   st.subheader("Please ")


       


if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
!ngrok authtoken 2GE9pLnERwJ5wZ1vHuGitlRjoD5_327yPZExMjvGdWqRQhQyK

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')

INFO:pyngrok.process:ngrok process starting: 15959
2023-03-19 22:15:43.265 ngrok process starting: 15959
INFO:pyngrok.process:t=2023-03-19T22:15:43+0000 lvl=info msg="no configuration paths supplied"

2023-03-19 22:15:43.288 t=2023-03-19T22:15:43+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2023-03-19T22:15:43+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2023-03-19 22:15:43.295 t=2023-03-19T22:15:43+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2023-03-19T22:15:43+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2023-03-19 22:15:43.301 t=2023-03-19T22:15:43+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

INFO:pyngrok.process:t=2023-03-19T22:15:43+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2023-03-19 22:15:43.306 t=2023-03-19T22:15:43+0000 lvl=info msg="starting

In [ ]:
!streamlit run app.py & npx localtunnel — port 8501

[############......] - refresh-package-json:localtunnel: timing action:finalize

npx: installed 22 in 5.169s
Usage: lt --port [num] <options>

Options:
  -p, --port                Internal HTTP server port                 [required]
  -h, --host                Upstream server providing forwarding
                                             [default: "https://localtunnel.me"]
  -s, --subdomain           Request this subdomain
  -l, --local-host          Tunnel traffic to this host instead of localhost,
                            override Host header to this host
      --local-https         Tunnel traffic to a local HTTPS server     [boolean]
      --local-cert          Path to certificate PEM file for local HTTPS server
      --local-key           Path to certificate key file for local HTTPS server
      --local-ca            Path to certificate authority file for self-signed
                            certificates
      --allow-invalid-cert  Disable certificate checks for your local

INFO:pyngrok.process:t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=3da8ad5c1efd l=127.0.0.1:8501 r=98.49.206.189:53051

2023-03-19 22:16:00.369 t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=3da8ad5c1efd l=127.0.0.1:8501 r=98.49.206.189:53051

INFO:pyngrok.process:t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=1c150385e776 l=127.0.0.1:8501 r=98.49.206.189:53050

2023-03-19 22:16:00.531 t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=1c150385e776 l=127.0.0.1:8501 r=98.49.206.189:53050

INFO:pyngrok.process:t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=d922ed20e0d6 l=127.0.0.1:8501 r=98.49.206.189:53067

2023-03-19 22:16:00.591 t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=d922ed20e0d6 l=127.0.0.1:8501 r=98.49.206.189:53067

INFO:pyngrok.process:t=2023-03-19T22:16:00+0000 lvl=info msg="join connections" obj=join id=63bdfeeb595d l=127.0.0.1:8501

2023-03-19 22:16:03.675351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 22:16:03.676220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 22:16:03.676540: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2

INFO:pyngrok.process:t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=c1d7c683f946 l=127.0.0.1:8501 r=98.49.206.189:53066

2023-03-19 22:20:32.636 t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=c1d7c683f946 l=127.0.0.1:8501 r=98.49.206.189:53066

INFO:pyngrok.process:t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=c9316e9b7683 l=127.0.0.1:8501 r=98.49.206.189:53051

2023-03-19 22:20:32.792 t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=c9316e9b7683 l=127.0.0.1:8501 r=98.49.206.189:53051

INFO:pyngrok.process:t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=b36b19c09ebe l=127.0.0.1:8501 r=98.49.206.189:53065

2023-03-19 22:20:32.798 t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=b36b19c09ebe l=127.0.0.1:8501 r=98.49.206.189:53065

INFO:pyngrok.process:t=2023-03-19T22:20:32+0000 lvl=info msg="join connections" obj=join id=1a08598ab6d0 l=127.0.0.1:8501

  Stopping...


In [ ]:
public_url

'http://1517-34-125-241-125.ngrok.io'